### Outline

0. Load Data and Restore results
<br />
<br />
1. Stabilization for linear transformations
<br />
<br />
2. Import packages and Load data
<br />
<br />
3. QC and preprocessing
<br />
<br />
4. Function: identify spatially variable genes
<br />
<br />
5. Function: gene expression enhancement
<br />
<br />
6. Function: characterize functional tissue units

In [1]:
import platform
platform.python_version()

'3.8.20'

## Step 0：Load Data and Restore results
Apply SpaGFT on the loaded data and compare SVG list output with the SVG list in the atlas 

SpaGFT is a python package to analyze spatial transcriptomics data via graph Fourier transform. To install SpaGFT, the python version is required to be >= 3.7. You can check your python version by:

### 0.1 Update me - global parameters

Global Parameters - Update all of them before running


In [37]:
# Naming
PATH_TO_DATA = r'data/stimage_data_downloads' # data main folder, divided to 3 subfolder: image, gene_exp and coord.
PATH_TO_RESULTS = './results/stimage_data/'
COORDS_COLUMNS = ['yaxis', 'xaxis'] # name of coordinates columns (e.g. ['x', 'y', 'z'], ['X', 'Y'])
CELL_COLUMN = 'cell' # name of cell ids column
GENE_COLUMN = 'gene' # name of gene columns
TISSUE_NAME = 'Human_Breast_Andersson_10142021_ST_A1' # suit to the tissue name in data file

# Scaling
CELL_DIAMETER = 7 # For plotting

# parameters for functional analysis
ORGANISM = 'Human'
GENE_SETS = ['BioPlanet_2019','GO_Biological_Process_2021']

In [16]:
import os
import pandas as pd
if not os.path.exists(PATH_TO_RESULTS):
    os.makedirs(PATH_TO_RESULTS)
results_folder = PATH_TO_RESULTS
coord_mtx = pd.read_csv(os.path.join(PATH_TO_DATA, 'coord', TISSUE_NAME + '_coord.csv'))
count_mtx = pd.read_csv(os.path.join(PATH_TO_DATA, 'gene_exp', TISSUE_NAME + '_count.csv'))


In [17]:
if 'Unnamed: 0' in coord_mtx.columns:
    coord_mtx.index = coord_mtx.iloc[:,0]
    coord_mtx.index.name = 'cell_id'
    coord_mtx.drop(columns='Unnamed: 0', inplace=True)
if 'Unnamed: 0' in count_mtx.columns:
    count_mtx.index = count_mtx.iloc[:,0]
    count_mtx.index.name = 'cell_id'
    count_mtx.drop(columns='Unnamed: 0', inplace=True)


### 2. Import packages and Load data

In [6]:
import SpaGFT as spg
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt


sc.settings.verbosity = 1
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.9.2 umap==0.5.7 numpy==1.21.5 scipy==1.10.1 pandas==1.4.2 scikit-learn==1.0.2 statsmodels==0.14.1 python-igraph==0.9.10 louvain==0.7.1 pynndescent==0.5.13


Two key elements are essential for running SpaGFT: the raw gene counts matrix and spatial coordinates of spots. Indeed, the basic object input to SpaGFT should be an anndata object which contains the above two elements.

You can also create your datasets which are ```anndata``` objects. Note that the raw count matrix should be found in adata.X, and the spatial coordinates of all spots should be found in ```adata.obs``` or ```adata.obsm```. You can use diverse functions provided by scanpy to create the anndata object. 
More approaches to load datasets can be found at [Scanpy](https://scanpy.readthedocs.io/en/stable/api.html#reading)


In [19]:

# create the anndata object
adata = sc.AnnData(count_mtx)
adata.obs.loc[:, COORDS_COLUMNS] = coord_mtx
adata.obsm = {'spatial': coord_mtx[COORDS_COLUMNS].values} 
adata.var_names_make_unique()
adata.raw = adata

### 3. QC and preprocessing
Before the formal analysis, we perform some basic filtering of genes and normalize Visium counts data with the ```normalize_total``` method from Scanpy followed by log-transform.

In [20]:
# QC
sc.pp.filter_genes(adata, min_cells=10)
# Normalization
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

### 4. Function: identify spatially variable genes

#### 4.1 Determine the Fourier modes and identify SVGs

To begin with, determine the number of Fourier modes for detecting SVGs. SpaGFT provides the dermine_frequency_ratio function based on the kneedle algorithm to determine how many FMs used automatically.
Note that *ratio_low * $\sqrt{n}$* and *ratio_high * $\sqrt{n}$* are the recommended low-frequency and high-frequency FMs, where *$\sqrt{n}$* is the number of spots in the original dataset.

In [21]:
# determine the number of low-frequency FMs and high-frequency FMs
(ratio_low, ratio_high) = spg.gft.determine_frequency_ratio(adata,
                                                            ratio_neighbors=1,
                                                            spatial_info=COORDS_COLUMNS)

Obatain the Laplacian matrix


In the following, SpaGFT will transform gene expression signals from the spatial domain to the frequency domain and analyze the frequency signals to identify SVGs.

In [22]:
# calculation
gene_df = spg.detect_svg(adata,
                         spatial_info=COORDS_COLUMNS,
                         ratio_low_freq=ratio_low,
                         ratio_high_freq=ratio_high,
                         ratio_neighbors=1,
                         filter_peaks=True,
                         S=6)
# S determines the  sensitivity of kneedle algorithm
# extract spaitally variable genes
svg_list = gene_df[gene_df.cutoff_gft_score][gene_df.fdr < 0.05].index.tolist()
print("The number of SVGs: ", len(svg_list))
# the top 20 SVGs
print("The top 20 SVGs:")
print(svg_list[:20])

# save the gene ranking for all genes
gene_df.to_csv(os.path.join(results_folder, TISSUE_NAME + '_gene_rank_gft.csv'))


The precalculated low-frequency FMs are USED
The precalculated high-frequency FMs are USED
Graph Fourier Transform finished!
svg ranking could be found in adata.var['svg_rank']
The spatially variable genes judged by gft_score could be found 
          in adata.var['cutoff_gft_score']
Gene signals in frequency domain when detect svgs could be found
          in adata.varm['freq_domain_svg']
The number of SVGs:  897
The top 20 SVGs:
['IGKC', 'TFF3', 'MUCL1', 'IGFBP2', 'APOE', 'TPT1', 'SERF2', 'MUC1', 'SCD', 'TAGLN2', 'IGHG3', 'KDELR1', 'KRT17', 'LMNA', 'FNBP1L', 'H2AFJ', 'PERP', 'SCGB2A2', 'COPE', 'ADAM15']


## Step 1：Stabilization for linear transformations